In [224]:
# Dependencies
from bs4 import BeautifulSoup as bs4
import requests

In [225]:
url = 'https://mars.nasa.gov/news'

In [226]:
# Retrieve page with the requests module
response = requests.get(url)

In [227]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [228]:
#print(soup.body.prettify())

In [235]:
results = soup.find_all('div', class_="image_and_description_container")
#results

In [236]:
results = soup.find_all('div', class_="content_title")
news_title = results[0].text

results = soup.find_all('div', class_="rollover_description_inner")
news_p = results[0].text

print (news_title)
print (news_p)

# the site seems to be malformed i was not able to scrape the latest article that is for Dec 10, when i looked at the 
# source code the first item that comes up is the one from August



NASA Invites Students to Name Mars 2020 Rover



Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.



In [237]:
from splinter import Browser
from bs4 import BeautifulSoup as bs

def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    #executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    executable_path = {"executable_path": "C:/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)


In [248]:
def scrape_featured_image():
    browser = init_browser()

    # Visit visitcostarica.herokuapp.com
    url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    base_url = "https://www.jpl.nasa.gov"
    browser.visit(url)
    
    html = browser.html
    soup = bs(html, "html.parser")
    
    button = browser.find_by_id('full_image')
    button.click()
    
    html = browser.html
    soup = bs(html, "html.parser")
    
    button = browser.find_link_by_partial_text('more info')
    button.click()
    
    html = browser.html
    soup = bs(html, "html.parser")
    
    # Interact with elements
    relative_image_path = soup.find_all('img', class_="main_image")[0]["src"]
    img = base_url + relative_image_path

    
    browser.quit()
    
    return img
    
img_url = scrape_featured_image()
print(img_url)    

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA14934_hires.jpg


In [134]:
response = requests.get("https://twitter.com/marswxreport?lang=en")
soup = BeautifulSoup(response.text, 'html.parser')
results = soup.find_all('div', class_="js-tweet-text-container")[0]
p = results.find('p').text
print(p)

InSight sol 372 (2019-12-13) low -97.2ºC (-143.0ºF) high -21.2ºC (-6.2ºF)
winds from the SSE at 5.9 m/s (13.3 mph) gusting to 20.2 m/s (45.2 mph)
pressure at 6.60 hPapic.twitter.com/SXXKNyUaJu


In [175]:
response = requests.get("https://space-facts.com/mars/")
soup = BeautifulSoup(response.text, 'html.parser')
results = soup.find_all('tr')


In [185]:
import pandas as pd
mars_facts = []
i = 0

# Loop through returned results
while 1:
    # Error handling
    try:
        measure = results[i].find_all('td')[0].text
        value = results[i].find_all('td')[1].text
        mars_facts.append({"measure":measure, "value":value})
        i = i + 1
    except IndexError as e:
        break

df = pd.DataFrame(mars_facts)
html = df.to_html()
print(html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>measure</th>
      <th>value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
      <t

In [223]:
def scrape_mars_images():
    browser = init_browser()

    url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
    base_url = "https://astrogeology.usgs.gov"
    browser.visit(url)
    
    html = browser.html
    soup = bs(html, "html.parser")
    
    i = 0
    
    titles = []
    hemisphere_image_urls = []
    
    #find number of images
    count = soup.find_all('div', class_='accordian')[0].find('span').text.split()[0]
    count = int(count)
    
    while i < count:
        title = soup.find_all('div', class_='collapsible results')[0].find_all('h3')[i].text
        titles.append(title)
        i = i + 1
    
    #print (titles)
    
    i = 0
    
    while i < count:
        link = browser.find_link_by_partial_text('Enhanced')[i]
        link.click()
        
        html = browser.html
        soup = bs(html, "html.parser")
        
        relative_image_path = soup.find_all('img', class_='wide-image')[0]["src"]
        img_url = base_url + relative_image_path
        
        #print (img_url)
        
        hemisphere_image_urls.append({"title":titles[i], "img_url":img_url})
        
        browser.back()
        
        i = i + 1
        
    browser.quit()

    return hemisphere_image_urls
    
hemisphere_image_urls = scrape_mars_images()
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]